In [ ]:
import polars   as pl
import networkx as nx
import random
import copy
import rtsvg
from rtsvg.polars_spring_layout import PolarsSpringLayout
rt = rtsvg.RACETrack()
_base_network_ = '0'
_base_dir_     = '../../data/stanford/facebook/'
_layout_file_  = _base_dir_ + _base_network_ + '.layout.parquet'
_edges_ = open(_base_dir_ + _base_network_ + '.edges', 'rt').read()
_lu_ = {'fm':[], 'to':[]}
for _edge_ in _edges_.split('\n'):
    if _edge_ == '': continue
    _lu_['fm'].append(_edge_.split(' ')[0])
    _lu_['to'].append(_edge_.split(' ')[1])
df        = pl.DataFrame(_lu_)
_relates_ = [('fm','to')]
g         = rt.createNetworkXGraph(df, _relates_)
pos       = rt.treeMapGraphComponentPlacement(g, PolarsSpringLayout(g).results())
_to_fix_  = {'181', '195', '218', '273', '275', '306', '328', '4', '78'}
_colors_  = {n:'red' for n in _to_fix_}
new_pos   = copy.deepcopy(pos)
for n in _to_fix_: new_pos[n] = (new_pos[n][0] + 100*(random.random()-0.5), new_pos[n][1] + 100*(random.random()-0.5))

new_new_pos = copy.deepcopy(new_pos)
new_new_pos = PolarsSpringLayout(g, static_nodes=set(g.nodes()) - _to_fix_).results()

rt.tile([rt.link(df, _relates_, pos,         w=512, h=512, node_color=_colors_),
         rt.link(df, _relates_, new_pos,     w=512, h=512, node_color=_colors_),
         rt.link(df, _relates_, new_new_pos, w=512, h=512, node_color=_colors_)], spacer=10)